# Алгоритмы интеллектуальной обработки больших объемов данных
## Семинар, учимся настраивать параметры бустинга


Разберем, например, xgboost:
https://xgboost.readthedocs.io/en/latest/parameter.html

https://xgboost.readthedocs.io/en/latest/python/python_api.html

Параметры можно разделить на группы. 



1) Общие параметры алгоритма, например, тип бустинга

2) Параметры обучения бустинга, например число итераций, learning rate

3) Параметры построения деревьев, например, глубина, минимальное число элементов в листе

4) Параметры стохастики, subsample, colsample

5) Регуляриазция, например, lambda


ПРАВИЛЬНОГО алгоритма для тюнинга нет, вот один из вариантов, которые часто используют на практике.

1) Возьмите дефолтное параметры. Поставьте нужные loss и eval функции.  Измените максимальное  число деревьев, чтобы было не долго делать эксперименты

2) Подберите для него alpha, чтобы алгоритм не переобучался

3) Подберите параметры сложности дерева, например,  глубину, мин число объектов в листах, мин значение для расщепления и тд

4) Подберите паарметры стохастики

5) Подберите регуляризацию

6) Уменьшите leraning rate и обучите максимальное число деревьев, чтобы не переобучалось

Давайте настроим параметры бустинга на нашем конкурсе!

In [67]:
doc_to_title = {}
with open('data/docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))


28026


In [68]:
import pandas as pd
train_data = pd.read_csv('data/train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

In [69]:
import numpy as np
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:15]    )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11690, 15) (11690,) (11690,)


Подберите размер батча для обучения. Линейная модель не должна учиться дольше нескольких минут. 

Не забывайте использовать скейлер!

In [4]:
from sklearn.metrics import f1_score, roc_auc_score
from tqdm import tqdm

In [32]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
import random
clf = xgb.XGBClassifier()
clf = lgb.LGBMClassifier()

In [6]:
from sklearn.linear_model import LogisticRegression

In [70]:
groups_train = groups_train.reshape(-1, 1)
uniq_for_shuffle = np.unique(groups_train)
random.shuffle(uniq_for_shuffle)

In [74]:
def cross_val_learner(**params):
    scores = []
    splits = 5
    val_gr_count = round(len(uniq_for_shuffle)/splits)

    for i in range(splits):
        val_groups = uniq_for_shuffle[i*val_gr_count:(i+1)*val_gr_count]
        val_idx = np.where(groups_train == val_groups)[0]
        train_groups = np.hstack((uniq_for_shuffle[:i*val_gr_count], uniq_for_shuffle[(i+1)*val_gr_count:]))
        train_idx = np.where(groups_train == train_groups)[0]

        X_tr = X_train[train_idx]
        y_tr = y_train[train_idx]
        X_val = X_train[val_idx]
        y_val = y_train[val_idx]
        
        xgbc = xgb.XGBClassifier(**params)
        xgbc.fit(X_tr, y_tr)
        y_pred = xgbc.predict(X_val)
        score = f1_score(y_val, y_pred)
        scores.append(score)
    return np.mean(scores)

### Подберем learning rate для n_estimators=30

In [75]:
learning_rate = np.linspace(0, 1, 100)
best_score = 0
best_eta = learning_rate[0]
bar = tqdm(total=learning_rate.shape[0])

for lr in learning_rate:
    score = cross_val_learner(eta=lr, n_estimators=30)
    if score > best_score:
        best_score = score
        best_eta = lr
    bar.update(1)



  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


  1%|          | 1/100 [00:00<01:02,  1.59it/s]

  2%|▏         | 2/100 [00:01<01:01,  1.61it/s]

  3%|▎         | 3/100 [00:01<01:02,  1.56it/s]

  4%|▍         | 4/100 [00:02<01:00,  1.59it/s]

  5%|▌         | 5/100 [00:03<00:58,  1.63it/s]

  6%|▌         | 6/100 [00:03<00:56,  1.65it/s]

  7%|▋         | 7/100 [00:04<00:56,  1.65it/s]

  8%|▊         | 8/100 [00:04<00:55,  1.67it/s]

  9%|▉         | 9/100 [00:05<00:58,  1.55it/s]

 10%|█         | 10/100 [00:06<01:00,  1.48it/s]

 11%|█         | 11/100 [00:06<00:57,  1.54it/s]

 12%|█▏        | 12/100 [00:07<00:55,  1.59it/s]

 13%|█▎        | 13/100 [00:08<00:56,  1.55it/s]

 14%|█▍        | 14/100 [00:09<01:00,  1.42it/s]

 15%|█▌        | 15/100 [00:10<01:10,  1.21

In [76]:
best_score, best_eta

(0.5525982501200873, 0.27272727272727276)

### Настроим параметры дерева

In [95]:
params = {'n_estimators': 30, 'eta': best_eta, 'scale_pos_weight': 2.5}
best_score = 0
gammas = np.logspace(-3, 2, 10)
min_child_weights = np.logspace(-3, 2, 10)

for i, g in enumerate(gammas):
    for min_ch_w in min_child_weights:
        for depth in range(1, 8):
            score = cross_val_learner(max_depth=depth, gamma=g, min_child_weight=min_ch_w, **params)
            if score > best_score:
                best_score = score
                best_depth = depth
                best_gamma = g
                best_mcw = min_ch_w

In [96]:
best_score, best_depth, best_gamma, best_mcw

(0.62545778149741, 7, 0.003593813663804626, 100.0)

### Параметры стохастики

In [92]:
params = {'n_estimators': 30, 'eta': best_eta, 'scale_pos_weight': 2.5, 'max_depth': 1}
best_score = 0
subsamples = np.linspace(0.25, 1, 4)
colsample_bytrees = np.linspace(0.25, 1, 4)
colsample_bynodes = np.linspace(0.25, 1, 4)

for subs in subsamples:
    for colt in colsample_bytrees:
        for coln in colsample_bynodes:
            score = cross_val_learner(subsample=subs, colsample_bytree=colt,\
                                      colsample_bynode=coln, **params)
            if score > best_score:
                best_score = score
                best_subs = subs
                best_colt = colt
                best_coln = coln

In [93]:
best_score, best_subs, best_colt, best_coln

(0.6241772064068352, 0.25, 1.0, 1.0)

### Регуляризация

In [97]:
params = {'n_estimators': 30, 'eta': best_eta, 'scale_pos_weight': 2.5,\
         'max_depth': 1, 'subsample': 0.25}
best_score = 0
lambdas = np.logspace(-2, 2, 40)

for l in lambdas:
    score = cross_val_learner(reg_lambda=l, **params)
    if score > best_score:
        best_score = score
        best_lambda = l

In [99]:
best_score, best_lambda

(0.625096291025729, 2.2854638641349907)

### Сделаем предикт на тесте

In [114]:
import pandas as pd
test_data = pd.read_csv('data/test_groups.csv')
testgroups_titledata = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))

In [115]:
X_test = []
groups_test = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_test.append(sorted(all_dist, reverse=True)[0:15]    )
X_test = np.array(X_test)
groups_test = np.array(groups_test)
print (X_test.shape, groups_test.shape)

(16627, 15) (16627,)


In [118]:
params = {'n_estimators': 30, 'eta': best_eta, 'scale_pos_weight': 2.5,\
         'max_depth': 1, 'subsample': 0.25, 'reg_lambda': best_lambda}
xgbc = xgb.XGBClassifier(**params)
xgbc.fit(X_train, y_train)
y_test = xgbc.predict(X_test)

In [119]:
len_test = y_test.shape[0]
idx = np.arange(11691, 11691+len_test)
y_test = y_test.reshape(-1, 1)
idx = idx.reshape(-1, 1)
y_test = np.hstack((idx, y_test))

In [120]:
submit = pd.DataFrame(y_test, columns=('pair_id', 'target'))
submit.to_csv('xgb.csv', index=False)

Задача: обучите на этих фичах (число фичей рассатриваем как гипераматр) бустинг, подберите параметры, засабмитьте на кагл. Побейте 0.62 паблик скор за сутки.  Удачи!